In [ ]:
x = open('./data/wiki_pos_tokenizer_with_taginfo.txt','r')

In [ ]:
listis = []
while 1:
    line = x.readline()
    if not line:break
    c = line.split()
    listis += list(map(lambda x: x.split('/')[0], c))

In [ ]:
keywords = []
for rawdata in useCollection.find({'site':'daum'}):
    if rawdata['keywords'] != 'NaN':
        keywords += rawdata['keywords']

In [ ]:
reKeywords = []
for i in keywords:
    if '·' in i:
        if '이슈' in i:
            reKeywords.append(i.split('·')[1].strip())
        else:
            reKeywords.append(i.strip())
    else:
        reKeywords.append(i.strip())

In [ ]:
tagger.add_dictionary(reKeywords, 'Noun')

In [134]:
# -*- coding: utf-8 -*-
import sys
sys.path.append('/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/')
from collections import defaultdict
from datetime import datetime, timedelta
import time
import re
import requests
import pandas as pd
import pickle
import chat_bot as cb
import Database_Handler as dh
from multiprocessing import Pool
from functools import partial
import pickle
import urllib3
import json
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Mecab, Twitter
from konlpy.utils import pprint
import nltk
from collections import Counter
from __future__ import print_function, unicode_literals
from konlpy.utils import pprint
from gensim.models.word2vec import Word2Vec
import itertools
def OpenAPI():
    import pickle
    openapi = pickle.load(open('./etri_open_api_access_key', 'rb'))
    return openapi
def USE_ETRI_ANALYSIS(analysisCode, text):    
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    accessKey = OpenAPI().strip()
    requestJson = {
        'access_key' : accessKey,
        'argument' : {
            'text' : text,
            'analysis_code' : analysisCode
        }
    }
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )
    return json.loads(str(response.data,'utf-8'))

def USE_ETRI_INFO(fWord, fSenseId, sWord, sSenseId):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseWWN/WordRel"
    accessKey = OpenAPI().strip()
    firstWord = fWord
    firstSenseId = fSenseId
    secondWord = sWord
    secondSenseId = sSenseId
    requestJson = {
        "access_key": accessKey,
        "argument": {
        'first_word': firstWord,
        'first_sense_id': firstSenseId,
        'second_word': secondWord,
        'second_sense_id': secondSenseId
        }}
    http = urllib3.PoolManager()
    response = http.request("POST",openApiURL,headers={"Content-Type": "application/json; charset=UTF-8"},
    body=json.dumps(requestJson))
    return json.loads(str(response.data, 'utf-8'))

def USE_ETRI_RELATION(word):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseWWN/Word"
    accessKey = OpenAPI().strip()
    word = word
    requestJson = {
    "access_key": accessKey,
    "argument": {"word": word}}
    http = urllib3.PoolManager()
    response = http.request("POST",openApiURL,headers={"Content-Type": "application/json; charset=UTF-8"},
                            body=json.dumps(requestJson))
    return json.loads(str(response.data, 'utf-8'))

def Extract_Text_Info(idx):
    morp = list(map(lambda x: (x['lemma'], x['type']),idx['morp']))
    morp = list(filter(lambda x: x[1] in ['NNG', 'NNP','NNB','SL','SH','NH'], morp))
    wsd = list(map(lambda x: (x['text'], x['type']),idx['WSD']))
    wsd = list(filter(lambda x: x[1] in ['NNG', 'NNP','NNG','SL','SH','NH'], wsd))
    word = list(map(lambda x: (x['text'], x['type']),idx['word']))
    ne = list(map(lambda x: (x['text'], x['type']),idx['NE']))
    return morp, wsd, word, ne

def TfidFeatures(pos,stopwords):
    posList = list(map(lambda x: x[0], pos))
    vect = TfidfVectorizer(stop_words=stopwords).fit(posList)
    count = vect.transform(posList).toarray().sum(axis = 0)
    idx = np.argsort(-count)
    feature_name = np.array(vect.get_feature_names())[idx]
    #feature_pos = np.array(list(map(lambda x:x[1], pos)))[idx]
    x = list(zip(feature_name, count))[:3]
    x = pd.DataFrame(x)
    x.set_index(0, inplace=True)
    return x
def Analysis(stopwordsList, text):
    #twitter_out = Twitter().pos(text)
    #mecab_out = Mecab().pos(text)
    #mecab_out = list(filter(lambda x: x[1] in ['NNB', 'NNP','NNG','SL','SH'], mecab_out))
    #twitter_out = list(filter(lambda x: x[1] in ['Foreign','Unknown','Noun'], twitter_out))
    etri = USE_ETRI_ANALYSIS('srl', text)['return_object']    
    etri = etri['sentence']
    morp = [] ; wsd = [] ; word = [] ; ne = [] ; srl = []
    for i in etri:
        x = Extract_Text_Info(i)
        morp += x[0]
        wsd += x[1]
        word +=x[2]
        ne += x[3]
        
    return morp, wsd, word, ne
    #eriOut_srl = TfidFeatures(srl, stopwords)
    #etriOut_morp = TfidFeatures(morp, stopwords)
    #mecabOut = TfidFeatures(mecab_out, stopwords)
    #twitterOut = TfidFeatures(twitter_out, stopwords)
    #etriOut_ne = TfidFeatures(ne, stopwords)
    #etriOut_wsd = TfidFeatures(wsd, stopwords)
    #etriOut_word = TfidFeatures(word, stopwords)
    #k = []
    #for i in etri:
    #    xx = i['dependency']
    #    k +=list(map(lambda x: (x['text'], x['label']), xx))
    #k = list(itertools.chain.from_iterable(list(map(lambda x: Twitter().pos(x[0]), k))))
    #k = list(filter(lambda x: x[1] in ['Foreign','Unknown','Noun'], k))
    #k = TfidFeatures(k, stopwords)
    #z = pd.concat([etriOut_morp, etriOut_ne,etriOut_word, mecabOut, twitterOut, k],axis = 1)
    #z1 = z.loc[:,z.columns.values==1].sum(axis = 1).sort_values()[-5:]
    #return etriOut_morp,eriOut_srl, etriOut_ne, etriOut_wsd, etriOut_word, mecabOut, twitterOut, k, z1

def Stopwords(file):
    stopwords = open(file,'r',  encoding='utf-8').readlines()
    stopwords = list(map(lambda x:x.strip(), stopwords))
    return stopwords
    
if __name__ == '__main__':
    pass

In [135]:
def RunLRNounExtractor(text):
    verbose=False
    left_max_length=8
    right_max_length=4
    min_count=5
    word_extractor = WordExtractor()
    noun_extractor = LRNounExtractor(None, 
                                 verbose, 
                                 left_max_length, 
                                 right_max_length, 
                                 min_count, 
                                 word_extractor)
    sentences = Sentences(text)
    nouns = noun_extractor.train_extract(sentences, minimum_noun_score=0.5, min_count=50)
    return nouns


In [136]:
from ckonlpy.tag import Twitter as ctwitter

In [137]:
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize

In [138]:
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer, MaxScoreTokenizer, RegexTokenizer
from soynlp.noun import LRNounExtractor

In [139]:
class Sentences:
    def __init__(self, text):
        self.text = text
        self.length = 0
    def __iter__(self):
            for doc in self.text.split('. '):
                doc = doc.strip()
                if not doc:
                    continue
                for sent in doc.split(' '):
                    yield sent
    def __len__(self):
        if self.length == 0:
            for doc in self.text.split('. '):
                doc = doc.strip()
                if not doc:
                    continue
                self.length += len(doc.split(' '))
        return self.length

In [140]:
stopwords = Stopwords('./data/koreanStopwords.txt')

In [141]:
mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
slack = cb.Slacker(cb.slacktoken())
useCollection = dh.Use_Collection(useDb, 'newsDaum')

In [142]:
rawdata = useCollection.find_one({'category':"뉴스"})
rawdata

{'_id': ObjectId('5a2af415588c1376f0d67afe'),
 'category': '뉴스',
 'date': '2017.12.06',
 'keywords': ['당뇨병', '탄수화물', '식이섬유'],
 'link': 'http://v.media.daum.net/v/20171206100855949',
 'mainText': " 사람들마다 식사를 할 때 밥과 반찬을 먹는 순서가 다르다. 밥부터 먼저 먹는 사람이 있는가 하면, 반찬을 먼저 먹은 후에 밥을 먹는 식이다. 그런데 식사를 할 때 무엇을 먼저 먹는지에 따라 살이 빠지고, 당뇨병까지 예방할 수 있다. 방법은 쉽다. 본인의 한 끼 식사를 영양소별로 나눠 먹는 순서를 정하고, 정한 순서대로 섭취해 살을 빼는 다이어트법이다. 우선 식단 내 영양소를 식이섬유, 단백질, 탄수화물, 크게 세 가지로 나눈다. 식이섬유가 많은 식품에는 채소·과일, 단백질이 많은 식품에는 고기·생선, 탄수화물이 많은 식품에는 쌀·보리 같은 곡류가 있다. 이때 순서를 식이섬유→단백질→탄수화물 순으로 음식을 섭취하면 된다. 예를 들어 일반적인 한식을 먹는다고 하면 나물과 김치를 먼저 먹은 후 다음에 국이나 찌개를 먹는다. 그런 후에 육류나 생선류를 먹고, 마지막으로 밥을 먹는다. 먹는 순서를 식이섬유→단백질→탄수화물 순으로 바꾸면 혈당이 급격하게 상승하는 것을 막고, 빠른 포만감을 준다. 또한 식이섬유부터 먹으면, 단백질·탄수화물이 천천히 흡수돼 혈당이 크게 상승하지 못하게 한다. 혈당이 높아지면 모두 에너지로 쓰이지 못하고 남으면서 지방으로 바뀌고, 비만으로 이어진다. 또 식이섬유를 먼저 먹으면 포만감이 잘 들어, 이후 먹는 단백질과 탄수화물 섭취량이 줄어든다. 특히 혈당을 많이 높이는 탄수화물 섭취를 막아 효과적이다. 실제로 쌀밥을 먹기 전에 생선이나 육류를 먼저 섭취하면 혈당이 급격히 높아지는 것을 억제할 수 있다는 연구결과도 있다. 간사이전력 의학연구소의 야베 다이스케 부소장 등 연구팀은 제2형 당뇨병 환자 1

In [143]:
useCollection2 = dh.Use_Collection(useDb, 'newsNaver')

In [144]:
rawdata2 = useCollection2.find_one({'category':"정치"})
rawdata2

{'_id': ObjectId('5a303059588c135900450b14'),
 'category': '정치',
 'date': '2017.12.12',
 'link': 'http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=003&aid=0008336872&date=20171212&type=1&rankingSectionId=100&rankingSeq=29',
 'mainText': '【서울=뉴시스】김진아 기자 = 삼성그룹의 한국동계스포츠영재센터(영재센터) 후원을 강요한 혐의 등으로 재판에 넘겨진 \'비선실세\' 최순실씨의 조카 장시호씨가 6일 오후 서울 서초구 서울중앙지방법원에서 열린 1심 선고 공판에서 징역 2년6월을 선고받은 후 법원을 나서고 있다. 2017.12.06. bluesoda@newsis.com   법정 구속후 첫 출석…감정 북받쳐 동요 "崔, 영재센터 3차 후원·EBS 인사 개입도"  【서울=뉴시스】이혜원 기자 = 최순실(61)씨가 지난해 10월 국정농단 사태가 불거지기 직전 삼성에 한국동계스포츠영재센터 3차 후원을 요구하려 했다는 법정 증언이 나왔다.  11일 서울고법 형사13부(부장판사 정형식) 심리로 열린 이재용(49) 삼성전자 부회장 등 삼성그룹 전·현직 임원 5명의 뇌물공여 등 혐의 항소심 12차 공판에서 최씨의 조카 장시호(38)씨가 증인으로 나와 이 같은 정황을 밝혔다.  장씨의 진술에 따르면 이규혁(39) 전 영재센터 전무는 지난해 10월 중순께 삼성전자에서 영재센터 추가 후원을 해줄 수 있는지 문의하기 위해 이영국 제일기획 상무에게 연락했다. 이에 이 상무는 일주일 뒤 만나자고 했지만, 결국 만남은 불발됐다.  장씨는 "최씨의 지시에 따라 이 전 전무가 삼성에 연락한 것이냐"는 질문에 "그렇다"라며 "최씨가 다음연도 예산안을 보내라고 했다"고 진술했다.  이어 특검이 "국정농단 사태가 언론을 통해 드러나기 직전에 최씨가

In [145]:
mecab = Mecab()
ct = ctwitter()
ot = Twitter()

In [146]:
for raw in useCollection.find({'site':'daum'}):
    if len(raw['mainText'].strip()) !=0:
        x = RunLRNounExtractor(raw['mainText'])
        k = list(filter(lambda x: x[1]=='Noun',ct.pos(raw['mainText'])))
        t =list(map(lambda x: x[0][0], Counter(k).most_common(n=5)))
        print (list(x.keys()), raw['keywords'], t)
        break

used noun_predictor_sejong
['이름'] ['노홍철', '한명회', '한혜진', '종영', '편성시간'] ['이름', '한명회', '시간', '회', '출연자']


In [147]:
etri = Analysis(stopwords, rawdata['mainText'])

In [154]:
et0 = list(map(lambda x: x[0], etri[0]))
et1 = list(map(lambda x: x[0], etri[1]))
et2 = list(map(lambda x: x[0], etri[2]))
et3 = list(map(lambda x: x[0], etri[3]))

In [155]:
mcout = list(map(lambda x: x[0], mecab.pos(rawdata['mainText'])))

In [156]:
ctout = list(map(lambda x: x[0], ct.pos(rawdata['mainText'])))
otout = list(map(lambda x: x[0], ot.pos(rawdata['mainText'])))

In [157]:
vect = TfidfVectorizer().fit(ctout+otout+mcout+et0+et1+et2+et3)

In [158]:
y = [ctout, otout, mcout, et0, et1, et2, et3]
outdict = dict()
for i in y:
    count = vect.transform(et3).toarray().sum(axis = 0)
    idx = np.argsort(-count)
    count = count[idx]
    feature_name = np.array(vect.get_feature_names())[idx]
    out = dict(zip(feature_name, count))
    for ii in out:
        if not ii in outdict:
            outdict[ii] = out[ii]
        else:
            outdict[ii] +=out[ii]

In [159]:
from operator import itemgetter
sorted(outdict.items(), key = itemgetter(1), reverse=True)

[('탄수화물', 56.0),
 ('단백질', 49.0),
 ('생선', 42.0),
 ('육류', 28.0),
 ('쌀밥', 28.0),
 ('당뇨병', 21.0),
 ('인슐린', 14.0),
 ('1그룹', 14.0),
 ('코넬대', 11.37555212392725),
 ('인크레틴', 7.0),
 ('고등어', 7.0),
 ('비만', 7.0),
 ('부소장', 7.0),
 ('교수', 7.0),
 ('구이', 7.0),
 ('베이컨', 7.0),
 ('찌개', 7.0),
 ('고기', 7.0),
 ('김치', 7.0),
 ('나물', 7.0),
 ('제2형', 7.0),
 ('미국', 7.0),
 ('인르레틴', 7.0),
 ('보리', 7.0),
 ('계란', 7.0),
 ('췌장', 7.0),
 ('30', 7.0),
 ('4시간', 7.0),
 ('2배', 7.0),
 ('가지', 7.0),
 ('2그룹', 7.0),
 ('40', 7.0),
 ('한식', 7.0),
 ('소고기', 7.0),
 ('124명', 7.0),
 ('소장', 7.0),
 ('15분', 7.0),
 ('12명', 7.0),
 ('후식', 7.0),
 ('30분', 7.0),
 ('10명', 7.0),
 ('3배', 7.0),
 ('환자', 7.0),
 ('연구팀', 5.4639311498952612),
 ('식사법', 5.1847658232184042),
 ('완싱크', 5.1288462926465961),
 ('야베', 5.0598861269831925),
 ('의학연구소', 4.9497474683058318),
 ('간사이전력', 4.9497474683058318),
 ('다이스케', 4.8371016509851241),
 ('브라이언', 4.7639202036143793),
 ('거꾸로', 4.7029994002111435),
 ('예방', 0.0),
 ('영양', 0.0),
 ('연구소', 0.0),
 ('영향', 0.0),
 ('영양소', 0.0),
 ('소화

In [161]:
%%time
nouns = noun_extractor.train_extract(sentences, minimum_noun_score=0.3, min_count=100)
print('num nouns = %d' % len(nouns))

NameError: name 'noun_extractor' is not defined

In [ ]:
print (nouns)

In [ ]:
import glob

In [ ]:
for file in glob.glob('./data/wikidumps/text/AA/wiki_*'):
    sentences = Sentences(file)
    word_extractor.train(sentences)
    

In [ ]:
words = word_extractor.extract()

In [ ]:
words['식이섬유']

In [ ]:
def CountFeatures(pos):
    posList = list(map(lambda x: x[0], pos))
    vect = CountVectorizer().fit(posList)
    count = vect.transform(posList).toarray().sum(axis = 0)
    idx = np.argsort(-count)
    feature_name = np.array(vect.get_feature_names())[idx]    
    x = list(zip(feature_name, count))[:5]     
    return x

In [ ]:
for idx in useCollection.find({'category':"뉴스"}):
    f = idx['mainText'].strip()
    if len(f) ==0:
        t = f
    else:
        t = CountFeatures(Twitter().pos(f))
    print (t,idx['keywords'])

In [ ]:
aaa = etri = USE_ETRI_ANALYSIS('srl', idx['mainText'])['return_object']    
etri = aaa['sentence']
morp = [] ; wsd = [] ; word = [] ; ne = []; srl =[]
for i in etri:
    x = Extract_Text_Info(i)
    morp += x[0]
    wsd += x[1]
    word +=x[2]
    ne += x[3]
    srl += x[4]

In [ ]:
len(np.array(list(map(lambda x:x[1], srl))))

In [ ]:
eriOut_srl = TfidFeatures(srl, stopwords)

In [ ]:
iii3 = TfidFeatures(iii2, stopwords)



In [ ]:
from gensim.models import doc2vec
import sys
import multiprocessing
cores = multiprocessing.cpu_count()
#doc2vec parameters
vector_size = 300
window_size = 15
word_min_count = 2
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100
dm = 1 #0 = dbow; 1 = dmpv
worker_count = cores #number of parallel processes
modelfile = "./model/doc2vec.model"
word2vec_file = modelfile + ".word2vec_format"
#sentences=doc2vec.TaggedLineDocument()
#build voca 
doc_vectorizer = doc2vec.Doc2Vec(min_count=word_min_count, size=vector_size, alpha=0.025, min_alpha=0.025, seed=1234, workers=worker_count)
doc_vectorizer.build_vocab(rawdata['mainText'])

In [ ]:
etriOut_word

In [ ]:
xx2 = list(set(map(lambda x: x[0], etriOut_ne)) & set(map(lambda x: x[0], mecabOut)))
xx2

In [ ]:
list(filter(lambda x: x[0] in xx, twitterOut ))

In [ ]:
set(map(lambda x: x[0], etriOut_word)) & set(map(lambda x: x[0], twitterOut))

In [ ]:
twitterOut

In [ ]:
set1 = Counter(list(filter(lambda x: ('NNG' or 'NNP' or 'NNB') ==x[1] , morp))).most_common(n= 20)
set1 = set(map(lambda x: x[0][0], set1))
set2 = Counter(list(filter(lambda x: ('NNG' or 'NNP' or 'NNB') ==x[1] , mecab_out))).most_common(n= 20)
set2 = set(map(lambda x: x[0][0], set2))
set3 = Counter(list(filter(lambda x: ('Noun') ==x[1] , twitter_out))).most_common(n= 20)
set3 = set(map(lambda x: x[0][0], set3))
set1 & set2 & set3

In [ ]:
#-*- coding: utf-8 -*-
from gensim.models import doc2vec
import sys
import multiprocessing
reload(sys)
sys.setdefaultencoding('utf-8')
cores = multiprocessing.cpu_count()
#doc2vec parameters
vector_size = 300
window_size = 15
word_min_count = 2
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100
dm = 1 #0 = dbow; 1 = dmpv
worker_count = cores #number of parallel processes


inputfile = "./data/"
modelfile = "./model/doc2vec.model"
word2vec_file = modelfile + ".word2vec_format"
sentences=doc2vec.TaggedLineDocument(inputfile)
#build voca 
doc_vectorizer = doc2vec.Doc2Vec(min_count=word_min_count, size=vector_size, alpha=0.025, min_alpha=0.025, seed=1234, workers=worker_count)
doc_vectorizer.build_vocab(sentences)
# Train document vectors!
for epoch in range(10):
	doc_vectorizer.train(sentences)
	doc_vectorizer.alpha -= 0.002 # decrease the learning rate
	doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay
# To save
doc_vectorizer.save(modelfile)
doc_vectorizer.save_word2vec_format(word2vec_file, binary=False)

# sentiment analysis

In [ ]:
import os
import sys

In [ ]:
def read_data(filename):
    with codecs.open(filename, encoding = 'utf-8', mode='r') as f:
        data